In [1]:
import requests
from bs4 import BeautifulSoup
import time
import random
import os
import urllib
from urllib.request import urlopen
import pickle
import sys
import numpy as np
import pandas as pd
import elementpath
import re

In [2]:
# Define the user agents to rotate
user_agents = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.79 Safari/537.36',
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36'
]

In [3]:
# test on one page

base_url = 'http://kremlin.ru'
path = '/events/president/transcripts/interviews/page/1'

# Make a GET request to the desired URL with rotating user agents and sleep time
for i in range(len(user_agents)):
    headers = {'User-Agent': user_agents[i]}
    url = base_url + path
    response = requests.get(url, headers=headers)

    # Check the status code
    if response.status_code == 200:
        # Parse the HTML content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

# Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
    time.sleep(1 + 4 * random.random())
    i = (i + 1) % len(user_agents)
    headers = {'User-Agent': user_agents[i]}


In [4]:
for link in soup.find_all('a'):
    print(link.get('href'))

#sidebar
/
/events/president/news
/structure/president
/multimedia/video
/acts/news
/contacts
/search
None
/acts/bank
#
#clear
#
#clear
http://pravo.gov.ru/
https://www.may9.ru/
/events/president/news
/events/administration
/events/state-council
/events/security-council
/events/councils
#focus
/events/president/news
/events/president/transcripts
/events/president/trips
/events/president/letters
/structure/president
/events/president/transcripts
/events/president/transcripts/messages
/events/president/transcripts/statements
/events/president/transcripts/deliberations
/events/president/transcripts/speeches
/events/president/transcripts/comminity_meetings
/events/president/transcripts/press_conferences
/events/president/transcripts/interviews
/events/president/transcripts/articles
/events/president/transcripts/interviews/70350
/events/president/transcripts/interviews/70350
/events/president/transcripts/interviews/70350/videos
/events/president/transcripts/interviews/68571
/events/presiden

In [5]:
soup.find_all('a')[34].get('href')

'/events/president/transcripts/interviews'

In [6]:
interview_links = soup.find_all("a", href=lambda href: href and href.startswith(
    "/events/president/transcripts/interviews"))

In [7]:
#find all links in the soup
base_url = 'http://kremlin.ru'

# Find all links with an href attribute that starts with "/events/president/transcripts/messages"
statements_links = soup.find_all("a", href=lambda href: href and href.startswith(
    "/events/president/transcripts/interviews"))

# convert the messages_links to a set to remove duplicates
statements_links = set(statements_links)

# create an empty list to store the urls
urls = []

# loop through the messages_links to create the urls
for link in statements_links:
    parts = link["href"].split("/")
    last_part = parts[-1]
    if last_part.isdigit():
        url = base_url + link["href"]
        urls.append(url)

urls = set(urls)
for url in urls:
    print(url)


http://kremlin.ru/events/president/transcripts/interviews/62971
http://kremlin.ru/events/president/transcripts/interviews/62937
http://kremlin.ru/events/president/transcripts/interviews/63007
http://kremlin.ru/events/president/transcripts/interviews/62991
http://kremlin.ru/events/president/transcripts/interviews/64171
http://kremlin.ru/events/president/transcripts/interviews/62975
http://kremlin.ru/events/president/transcripts/interviews/68571
http://kremlin.ru/events/president/transcripts/interviews/63951
http://kremlin.ru/events/president/transcripts/interviews/66191
http://kremlin.ru/events/president/transcripts/interviews/67695
http://kremlin.ru/events/president/transcripts/interviews/62997
http://kremlin.ru/events/president/transcripts/interviews/62931
http://kremlin.ru/events/president/transcripts/interviews/62963
http://kremlin.ru/events/president/transcripts/interviews/67100
http://kremlin.ru/events/president/transcripts/interviews/70350
http://kremlin.ru/events/president/trans

In [8]:
len(urls)

21

## Loop over 8 pages and get the urls

In [9]:
base_url = 'http://kremlin.ru'
path = '/events/president/transcripts/interviews/page/'

In [10]:
urls = []

# Loop over 8 pages
for page_num in range(1, 10):
    url = base_url + path + str(page_num)
    headers = {'User-Agent': user_agents[page_num % len(user_agents)]}
    response = requests.get(url, headers=headers)

    # Check the status code
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        # TODO: Scrape the URLs from the page and save them to a list
        for link in soup.find_all('a'):
            href = link.get('href')
            if href and href.startswith('/'):
                urls.append(base_url + href)
                print(base_url + href)  # Print the URL while it works
    else:
        print(
            f'Request for {url} failed with status code {response.status_code}')

    # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
    time.sleep(1 + 4 * random.random())


http://kremlin.ru/
http://kremlin.ru/events/president/news
http://kremlin.ru/structure/president
http://kremlin.ru/multimedia/video
http://kremlin.ru/acts/news
http://kremlin.ru/contacts
http://kremlin.ru/search
http://kremlin.ru/acts/bank
http://kremlin.ru/events/president/news
http://kremlin.ru/events/administration
http://kremlin.ru/events/state-council
http://kremlin.ru/events/security-council
http://kremlin.ru/events/councils
http://kremlin.ru/events/president/news
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/trips
http://kremlin.ru/events/president/letters
http://kremlin.ru/structure/president
http://kremlin.ru/events/president/transcripts
http://kremlin.ru/events/president/transcripts/messages
http://kremlin.ru/events/president/transcripts/statements
http://kremlin.ru/events/president/transcripts/deliberations
http://kremlin.ru/events/president/transcripts/speeches
http://kremlin.ru/events/president/transcripts/comminity_meetings
http://kreml

In [11]:
len(urls)

963

In [12]:
# filter only the statements + digits
filtered_urls = [url for url in urls if url.startswith(
    'http://kremlin.ru/events/president/transcripts/interviews') and re.search(r'\d$', url)]


In [13]:
filtered_urls = set(filtered_urls)
for url in filtered_urls:
    print(url)

http://kremlin.ru/events/president/transcripts/interviews/20384
http://kremlin.ru/events/president/transcripts/interviews/22162
http://kremlin.ru/events/president/transcripts/interviews/5516
http://kremlin.ru/events/president/transcripts/interviews/643
http://kremlin.ru/events/president/transcripts/interviews/2312
http://kremlin.ru/events/president/transcripts/interviews/17723
http://kremlin.ru/events/president/transcripts/interviews/9888
http://kremlin.ru/events/president/transcripts/interviews/page/8
http://kremlin.ru/events/president/transcripts/interviews/47104
http://kremlin.ru/events/president/transcripts/interviews/6964
http://kremlin.ru/events/president/transcripts/interviews/22185
http://kremlin.ru/events/president/transcripts/interviews/7659
http://kremlin.ru/events/president/transcripts/interviews/66191
http://kremlin.ru/events/president/transcripts/interviews/8082
http://kremlin.ru/events/president/transcripts/interviews/22145
http://kremlin.ru/events/president/transcripts/

In [14]:
len(filtered_urls)

189

In [15]:
urls = list(filtered_urls)
len(urls)

189

In [16]:
# Save the list to a file
with open('data/corpus_adriana/corpus_president_of_russia/interviews_urls.pkl', 'wb') as f:
    pickle.dump(urls, f)

# Load the list from the file
with open('data/corpus_adriana/corpus_president_of_russia/interviews_urls.pkl', 'rb') as f:
    urls = pickle.load(f)

### Make a soup for each url in interviews_urls

In [17]:
# make a soup for each url of 168urls object

soup_list = []
processed_urls = []

for url in urls:
    # Preprocess the URL
    processed_url = url.strip()

    # Check if the URL has already been processed
    if processed_url in processed_urls:
        print(f"Skipping {processed_url} - already processed")
        continue

    try:
        # Make a GET request to the desired URL with rotating user agents and sleep time
        i = 0
        while True:
            headers = {'User-Agent': user_agents[i]}
            response = requests.get(processed_url, headers=headers, timeout=10)

            if response.status_code == 200:
                # make a soup object from the response content
                soup = BeautifulSoup(response.content, 'html.parser')
                soup_list.append(soup)

                # Sleep for a random time between 1 and 5 seconds before making the next request with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)
                processed_urls.append(processed_url)
                print(f"Processed URL: {processed_url}")
                break

            else:
                # If the response is not 200, sleep for a random time between 1 and 5 seconds and try again with a different user agent
                time.sleep(1 + 4 * random.random())
                i = (i + 1) % len(user_agents)

    except Exception as e:
        print(f"Error getting data from {processed_url}: {e}")


Processed URL: http://kremlin.ru/events/president/transcripts/interviews/20384
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/22162
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/5516
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/643
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/2312
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/17723
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/9888
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/page/8
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/47104
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/6964
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/22185
Processed URL: http://kremlin.ru/events/president/transcripts/interviews/7659
Processed URL: http://kremlin.ru/events/president/transcri

In [18]:
len(soup_list)

189

In [19]:
# filter out the url ending in page {digit}
filtered_interviews_soup_list = [soup for soup, url in zip(
    soup_list, urls) if not re.search(r'page/\d+$', url)]
len(filtered_interviews_soup_list)

180

In [20]:
# increase the recursion limit
sys.setrecursionlimit(10**6)

In [21]:
# Save the soup_list to a pickle file
with open('data/corpus_adriana/corpus_president_of_russia/filtered_interviews_soup_list.pkl', 'wb') as f:
    pickle.dump(soup_list, f)

### find elements in each soup of filtered_interviews_soup_list

In [62]:
# Load the soup_list from the pickle file
with open('data/corpus_adriana/corpus_president_of_russia/filtered_interviews_soup_list.pkl', 'rb') as f:
    soup_list = pickle.load(f)

In [63]:
filtered_interviews_soup_list = [soup for soup, url in zip(
    soup_list, urls) if not re.search(r'page/\d+$', url)]
len(filtered_interviews_soup_list)

180

In [64]:
soup_list = filtered_interviews_soup_list

In [39]:
# datetime
for soup in soup_list:
    print(soup.find('time', {'itemprop': 'datePublished'})['datetime'])

2014-03-06
2003-10-16
2009-09-20
2008-07-03
2008-12-04
2013-03-22
2010-12-24
2014-11-28
2010-02-25
2003-11-03
2010-05-07
2021-07-13
2010-06-18
2003-10-04
2008-09-02
2010-12-19
2013-06-14
2009-06-21
2008-05-22
2010-08-30
2008-12-24
2008-11-13
2015-02-23
2009-11-23
2013-09-04
2005-08-24
2019-09-02
2014-10-15
2019-04-25
2006-11-17
2016-06-23
2014-07-11
2004-12-03
2010-12-06
2010-04-26
2020-02-20
2014-11-24
2020-03-02
2012-09-06
2005-05-07
2017-04-12
2009-11-07
2003-05-27
2020-03-11
2020-03-05
2018-03-10
2020-02-21
2013-04-05
2011-12-24
2009-06-15
2018-06-04
2009-06-03
2006-10-10
2022-06-03
2020-08-27
2003-10-19
2006-10-10
2020-03-17
2020-03-04
2019-01-16
2016-10-13
2013-07-23
2011-09-09
2010-04-14
2010-04-12
2016-08-05
2008-07-03
2006-03-01
2006-07-12
2021-11-13
2014-11-14
2023-01-13
2010-11-09
2015-05-09
2014-04-19
2005-05-09
2020-03-13
2006-02-07
2005-09-14
2005-09-17
2009-04-19
2007-02-10
2010-09-26
2016-12-13
2015-10-12
2015-07-27
2008-08-31
2015-09-29
2014-06-04
2010-11-08
2020-02-27

In [40]:
# sorted date
sorted_soup_list = sorted(soup_list, key=lambda soup: soup.find(
    'time', {'itemprop': 'datePublished'})['datetime'])
for soup in sorted_soup_list:
    print(soup.find('time', {'itemprop': 'datePublished'})['datetime'])

2003-02-28
2003-04-02
2003-05-27
2003-06-03
2003-06-22
2003-07-03
2003-09-20
2003-10-04
2003-10-16
2003-10-19
2003-11-03
2004-09-01
2004-10-13
2004-12-03
2005-02-22
2005-04-20
2005-04-25
2005-05-07
2005-05-09
2005-08-24
2005-09-14
2005-09-17
2005-10-31
2006-02-07
2006-03-01
2006-03-20
2006-07-12
2006-07-12
2006-07-12
2006-07-13
2006-10-10
2006-10-10
2006-11-17
2006-11-23
2006-12-07
2007-01-18
2007-02-10
2007-06-04
2007-12-19
2008-05-22
2008-06-25
2008-07-03
2008-07-03
2008-08-31
2008-09-02
2008-09-02
2008-11-13
2008-12-04
2008-12-24
2009-01-30
2009-02-15
2009-03-01
2009-03-15
2009-03-29
2009-04-13
2009-04-19
2009-05-16
2009-06-03
2009-06-04
2009-06-15
2009-06-18
2009-06-21
2009-07-05
2009-08-07
2009-09-18
2009-09-20
2009-10-19
2009-11-07
2009-11-23
2010-02-25
2010-04-12
2010-04-14
2010-04-24
2010-04-26
2010-05-07
2010-05-16
2010-06-18
2010-08-30
2010-09-26
2010-11-08
2010-11-09
2010-11-11
2010-12-06
2010-12-19
2010-12-24
2011-01-26
2011-01-27
2011-04-12
2011-04-25
2011-06-20
2011-08-05

In [42]:
# url
for soup in sorted_soup_list:
    url = soup.find('div', {'class': 'read'}).get('data-href')
    print(f"{url}")

http://kremlin.ru/events/president/transcripts/interviews/21891
http://kremlin.ru/events/president/transcripts/interviews/21952
http://kremlin.ru/events/president/transcripts/interviews/22002
http://kremlin.ru/events/president/transcripts/interviews/24767
http://kremlin.ru/events/president/transcripts/interviews/22031
http://kremlin.ru/events/president/transcripts/interviews/22049
http://kremlin.ru/events/president/transcripts/interviews/22125
http://kremlin.ru/events/president/transcripts/interviews/22145
http://kremlin.ru/events/president/transcripts/interviews/22162
http://kremlin.ru/events/president/transcripts/interviews/22163
http://kremlin.ru/events/president/transcripts/interviews/22185
http://kremlin.ru/events/president/transcripts/interviews/22585
http://kremlin.ru/events/president/transcripts/interviews/22633
http://kremlin.ru/events/president/transcripts/interviews/22716
http://kremlin.ru/events/president/transcripts/interviews/22837
http://kremlin.ru/events/president/trans

In [43]:
## extract the title
for soup in soup_list:
    print(soup.find('title').text.strip())


Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России
Интервью телеканалу «Аль-Джазира» • Президент России
Интервью телекомпании «Си-Эн-Эн» • Президент России
Интервью представителям средств массовой информации стран «Группы восьми» • Президент России
Интервью индийскому телеканалу «Дурдаршан» • Президент России
Интервью информационному агентству ИТАР-ТАСС • Президент России
Итоги года с Президентом России • Президент России
Интервью турецкому информационному агентству «Анадолу» • Президент России
Интервью французскому журналу «Пари-матч» • Президент России
Интервью итальянским информационному агентству АНСА, газете «Коррьере делла сера» и телекомпании РАИ • Президент России
Интервью Дмитрия Медведева газете «Известия» • Президент России
Владимир Путин ответил на вопросы о статье «Об историческом единстве русских и украинцев» • Президент России
Интервью американской газете «Уолл-стрит джорнэл» • Президент России
Интервью газете «Нью-Йорк Таймс» • Президент 

In [46]:
#speaker
soup = soup_list[10]
for p in soup.find_all('p'):
    speaker_tag = p.find('b')
    if speaker_tag is not None:
        print(speaker_tag.text)

В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев

В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев
В.Абрамов
Д.Медведев:
В.Абрамов:
Д.Медведев:


In [50]:
# Find the soup
soup = soup_list[10]

# Extract the date and URL
datetime = soup.find('time', {'itemprop': 'datePublished'})['datetime']
url = soup.find('div', {'class': 'read'}).get('data-href')
    
# Extract the title
title = soup.find('title').text.strip()

# Find all <p> tags in the soup
p_tags = soup.find_all('p')

# Initialize the speaker variable to None
speaker = None
assign_speaker = False

# Initialize a list to store the data
data = []

# Loop through all <p> tags and assign a speaker if needed
for p_tag in p_tags:
    # Check if the <p> tag has a <b> tag inside it
    if p_tag.find('b'):
        # If it does, save the <b> tag as the current speaker
        speaker = p_tag.find('b').text.strip()
        assign_speaker = True
    else:
        # If it doesn't and a <b> tag has been detected, assign the last speaker to this <p> tag
        if assign_speaker and speaker is not None:
            # Create a new <b> tag with the speaker's name and insert it before the <p> tag
            new_tag = soup.new_tag('b')
            new_tag.string = speaker
            p_tag.insert(0, new_tag)

# Find all <p> tags
p_tags = soup.find_all('p')

# Extract the speaker's name and the corresponding text from each <p> tag
for p_tag in p_tags:
    # Check if the <p> tag has a <b> tag inside it
    if p_tag.find('b'):
        # Extract the speaker's name from the <b> tag
        speaker_name = p_tag.b.text.strip()

        # Extract the text from the <p> tag (excluding the <b> tag)
        text = ' '.join(p_tag.find_all(string=True, recursive=False)).replace(speaker_name, '', 1).strip()

        # Add the data to the list
        data.append({'date': datetime, 'url': url, 'title': title, 'speaker': speaker_name, 'text': text})

# Create a pandas DataFrame from the data
df = pd.DataFrame(data)

# Print the DataFrame
df

,date,url,title,speaker,text
0,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,В.Абрамов,: Уважаемый Дмитрий Анатольевич! В прежние год...
1,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев,": Вы знаете, я думаю, это была абсолютно точна..."
2,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев,"Мои родственники и по материнской, и по отцовс..."
3,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев,И дед Вениамин — он тоже мне довольно много ра...
4,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев,Мои родители оказались в эвакуации. Мама была ...
...,...,...,...,...,...
105,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев:,
106,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев:,Официальный сайт президента России:
107,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев:,Интервью Дмитрия Медведева газете «Известия»
108,2010-05-07,http://kremlin.ru/events/president/transcripts...,Интервью Дмитрия Медведева газете «Известия» •...,Д.Медведев:,http://kremlin.ru/events/president/transcripts...


In [60]:
pd.set_option('display.max_colwidth', 100)
df[['speaker', 'text']].tail(10)

,speaker,text
100,Д.Медведев:,"Спасибо вам большое. Я желаю успехов вам, успехов газете. И через газету хотел бы искренне поздр..."
101,Д.Медведев:,"Опубликован в разделах: ,"
102,Д.Медведев:,Дата публикации:
103,Д.Медведев:,Ссылка на материал:
104,Д.Медведев:,
105,Д.Медведев:,
106,Д.Медведев:,Официальный сайт президента России:
107,Д.Медведев:,Интервью Дмитрия Медведева газете «Известия»
108,Д.Медведев:,http://kremlin.ru/events/president/transcripts/interviews/7659
109,Д.Медведев:,Администрация Президента России 2023 год


In [79]:
# Initialize an empty list to store the data from all soups
all_data = []

# Loop through all soups in soup_list
for soup in soup_list:
    # Reset the speaker variable to None at the beginning of each soup
    speaker = None
    
    # Extract the date and URL
    datetime = soup.find('time', {'itemprop': 'datePublished'})['datetime']
    url = soup.find('div', {'class': 'read'}).get('data-href')
        
    # Extract the title
    title = soup.find('title').text.strip()

    # Find all <p> tags in the soup
    p_tags = soup.find_all('p')

    # Initialize a list to store the data for this soup
    data = []

    # Loop through all <p> tags and assign a speaker if needed
    for p_tag in p_tags:
        # Check if the <p> tag has a <b> tag inside it
        if p_tag.find('b'):
            # If it does, save the <b> tag as the current speaker
            speaker = p_tag.find('b').text.strip()
            assign_speaker = True
        else:
            # If it doesn't and a <b> tag has been detected, assign the last speaker to this <p> tag
            if assign_speaker and speaker is not None:
                # Create a new <b> tag with the speaker's name and insert it before the <p> tag
                new_tag = soup.new_tag('b')
                new_tag.string = speaker
                p_tag.insert(0, new_tag)

    # Find all <p> tags (including those that may have just been modified with a new <b> tag)
    p_tags = soup.find_all('p')

    # Extract the speaker's name and the corresponding text from each <p> tag
    for p_tag in p_tags:
        # Check if the <p> tag has a <b> tag inside it
        if p_tag.find('b'):
            # Extract the speaker's name from the <b> tag
            speaker_name = p_tag.b.text.strip()

            # Extract the text from the <p> tag (excluding the <b> tag)
            text = ' '.join(p_tag.find_all(string=True, recursive=False)).replace(speaker_name, '', 1).strip()

            # Add the data to the list for this soup
            data.append({'date': datetime, 'url': url, 'title': title, 'speaker': speaker_name, 'text': text})

    # Add the data for this soup to the list for all soups
    all_data.extend(data)

# Create a pandas DataFrame from the data for all soups
df = pd.DataFrame(all_data)


In [87]:
# save the dataframe
#save the df as pickle
df.to_pickle('data/corpus_adriana/corpus_president_of_russia/interviews.pkl')

In [88]:
df = pd.read_pickle('data/corpus_adriana/corpus_president_of_russia/interviews.pkl')
df

,date,url,title,speaker,text
0,2014-03-06,http://kremlin.ru/events/president/transcripts/interviews/20384,Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России,Вопрос:,"С точки зрения внимания общественности и СМИ, лондонская Паралимпиада – прорыв. По рейтингам и а..."
1,2014-03-06,http://kremlin.ru/events/president/transcripts/interviews/20384,Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России,В.Путин:,"Начнём с того, что Великобританию принято считать родиной Паралимпийских игр, ведь первыми спорт..."
2,2014-03-06,http://kremlin.ru/events/president/transcripts/interviews/20384,Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России,В.Путин:,Наша страна принимает участие в Паралимпиадах с восьмидесятых годов прошлого века и имеет хороши...
3,2014-03-06,http://kremlin.ru/events/president/transcripts/interviews/20384,Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России,В.Путин:,"В целом паралимпийский спорт, его замечательные представители привлекают всё больше внимания. Оч..."
4,2014-03-06,http://kremlin.ru/events/president/transcripts/interviews/20384,Интервью российским и иностранным СМИ в преддверии Паралимпиады • Президент России,В.Путин:,"Кроме того, мы будем масштабно транслировать эти соревнования. Международное телевизионное освещ..."
...,...,...,...,...,...
9637,2016-01-11,http://kremlin.ru/events/president/transcripts/interviews/51154,Интервью немецкому изданию Bild. Часть 1 • Президент России,В.Путин:,
9638,2016-01-11,http://kremlin.ru/events/president/transcripts/interviews/51154,Интервью немецкому изданию Bild. Часть 1 • Президент России,В.Путин:,Официальный сайт президента России:
9639,2016-01-11,http://kremlin.ru/events/president/transcripts/interviews/51154,Интервью немецкому изданию Bild. Часть 1 • Президент России,В.Путин:,Интервью немецкому изданию Bild. Часть 1
9640,2016-01-11,http://kremlin.ru/events/president/transcripts/interviews/51154,Интервью немецкому изданию Bild. Часть 1 • Президент России,В.Путин:,http://kremlin.ru/events/president/transcripts/interviews/51154
